In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 管道: 评估 AutoML 表格回归模型的批量预测结果

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本笔记本演示了如何使用Vertex AI回归模型评估组件来评估AutoML表格回归模型。模型评估可以帮助您根据评估指标确定模型的性能，并在必要时改进模型。

了解更多关于[Vertex AI模型评估](https://cloud.google.com/vertex-ai/docs/evaluation/introduction)。了解更多关于[表格数据回归](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)。

### 目标

在本教程中，您将学习如何通过使用 `google_cloud_pipeline_components` 在 Vertex AI 管道作业中评估 Vertex AI 模型资源：

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 数据集（表格）
- Vertex AI 训练（AutoML 表格训练）
- Vertex AI 批量预测
- Vertex AI 管道
- Vertex AI 模型注册表


执行的步骤包括：

- 创建 Vertex AI 数据集
- 配置 `AutoMLTabularTrainingJob`
- 运行 `AutoMLTabularTrainingJob`，返回一个模型
- 将预训练的 `AutoML 模型资源` 导入管道
- 在管道中运行 `批量预测` 作业
- 使用 `回归评估组件` 评估 AutoML 模型
- 将回归指标导入到 AutoML 模型资源中

### 数据集

这个笔记本中使用的数据集是PetFinder数据集的一部分，可以在Kaggle上的[这里](https://www.kaggle.com/c/petfinder-adoption-prediction)找到。当前数据集只是原始数据集的一部分，用于预测宠物年龄的问题。它包含以下字段：

- `Type`：动物类型（1 = 狗，2 = 猫）
- `Age`：宠物在列出时的年龄，以月计
- `Breed1`：宠物的主要品种
- `Gender`：宠物的性别
- `Color1`：宠物的颜色1
- `Color2`：宠物的颜色2
- `MaturitySize`：成熟时的大小（1 = 小型，2 = 中型，3 = 大型，4 = 特大型，0 = 未指定）
- `FurLength`：毛发长度（1 = 短毛，2 = 中毛，3 = 长毛，0 = 未指定）
- `Vaccinated`：宠物已接种疫苗（1 = 是，2 = 否，3 = 不确定）
- `Sterilized`：宠物已绝育（1 = 是，2 = 否，3 = 不确定）
- `Health`：健康状况（1 = 健康，2 = 轻伤，3 = 严重伤害，0 = 未指定）
- `Fee`：领养费用（0 = 免费）
- `PhotoAmt`：为这个宠物上传的照片总数
- `Adopted`：宠物是否被领养（是/否）。

**注意**：该数据集已移至一个公共云存储桶，并在该笔记本中从那里访问。

成本
本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-pipeline-components==1.0.26 \
                                 matplotlib 

只有Colab才能使用：请取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在开始之前

设置您的项目 ID

如果您不知道您的项目 ID，可以尝试以下方式：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"
DATA_REGION = "US"

### 验证您的Google Cloud账号

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经通过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请参阅如何将云存储权限授予您的服务账户 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

服务账号

您可以使用服务账号创建Vertex AI管道作业。如果您不想使用项目的计算引擎服务账号，请将`SERVICE_ACCOUNT`设置为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI流水线设置服务账号访问权限

运行以下命令以授予您的服务账号对在上一步中创建的存储桶中的管道工件进行读取和写入的访问权限。您只需要针对每个服务账号运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

In [ ]:
import json

import google.cloud.aiplatform as aiplatform
import matplotlib.pyplot as plt
from google.cloud import aiplatform_v1

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建顶点 AI 数据集

使用数据集来源在顶点 AI 中创建一个托管的表格数据集资源。

In [ ]:
DATA_SOURCE = "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/petfinder-tabular-classification.csv"

In [ ]:
# Create the Vertex AI Dataset resource
dataset = aiplatform.TabularDataset.create(
    display_name="petfinder-tabular-dataset",
    gcs_source=DATA_SOURCE,
)

print("Resource name:", dataset.resource_name)

训练AutoML模型

使用创建的数据集训练一个简单的回归模型，以`Age`作为目标列。

**设置显示名称并创建`AutoMLTabularTrainingJob`，指定适当的列转换数据类型。**

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "[your-train-job-display-name]"  # @param {type:"string"}

In [ ]:
# If no display name is specified, use the default one
if (
    TRAINING_JOB_DISPLAY_NAME == ""
    or TRAINING_JOB_DISPLAY_NAME is None
    or TRAINING_JOB_DISPLAY_NAME == "[your-train-job-display-name]"
):
    TRAINING_JOB_DISPLAY_NAME = "train-pet-agefinder-automl"

### 定义 AutoML 表格训练任务

使用 `AutoMLTabularTrainingJob` 类创建 AutoML 训练作业，具有以下参数：

- `display_name`：`TrainingJob` 资源的人类可读名称。
- `optimization_prediction_type`：AutoML 模型要生成的预测类型。例如：回归、分类。
- `column_transformations`：要应用于输入列（即目标列以外的列）的转换。每个转换可能会从列的值产生多个结果值，并且所有结果将用于训练。
- `optimization_objective`：要最小化或最大化的优化目标。
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

了解有关 [AutoMLTabularTrainingJob](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob) 的更多信息。

In [ ]:
train_job = aiplatform.AutoMLTabularTrainingJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    optimization_prediction_type="regression",
    column_specs={
        "Type": "categorical",
        "Breed1": "categorical",
        "Gender": "categorical",
        "Color1": "categorical",
        "Color2": "categorical",
        "MaturitySize": "categorical",
        "FurLength": "categorical",
        "Vaccinated": "categorical",
        "Sterilized": "categorical",
        "Health": "categorical",
        "Fee": "numeric",
        "PhotoAmt": "numeric",
        "Adopted": "categorical",
    },
    optimization_objective="minimize-rmse",
)

print(train_job)

设置模型的显示名称。

In [ ]:
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

In [ ]:
# If no name is specified, use the default name
if (
    MODEL_DISPLAY_NAME == ""
    or MODEL_DISPLAY_NAME is None
    or MODEL_DISPLAY_NAME == "[your-model-display-name]"
):
    MODEL_DISPLAY_NAME = "pet-agefinder-prediction-model"

运行训练作业

接下来，您可以通过调用`run`方法来启动训练作业，以下是参数：

- `dataset`：要训练模型的`Dataset`资源。
- `target_column`：模型要预测数值的列名。
- `training_fraction_split`：要用于训练的数据集百分比。
- `validation_fraction_split`：要用于验证的数据集百分比。
- `test_fraction_split`：要用于测试（留存数据）的数据集百分比。
- `model_display_name`：训练模型的人类可读名称。
- `budget_milli_node_hours`：创建此模型的训练预算，以毫节点小时表示，即该字段中的值为1,000表示1个节点小时。

训练作业大约需要3小时才能完成。

In [ ]:
# Run the training job
model = train_job.run(
    dataset=dataset,
    target_column="Age",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name=MODEL_DISPLAY_NAME,
    budget_milli_node_hours=1000,
)

## 列出来自训练的模型评估

训练作业完成后，获取模型评估并打印出来。

In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

In [ ]:
# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

运行一条管道进行模型评估

现在，您可以运行一个Vertex AI BatchPrediction作业，并通过使用`evaluate`函数创建一个Vertex AI管道生成评估和特征归因结果。了解更多关于[evaluate函数](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/models.py#L5127)。

###定义运行评估函数的参数

指定运行`evaluate`函数所需的参数。

以下是`evaluate`函数参数的说明：

- `prediction_type`：评估运行所处理的问题类型。目前支持的问题类型是“分类”和“回归”。
- `target_field_name`：用作回归目标的列的名称。
- `gcs_source_uris`：云存储桶URI列表，用于批量预测的输入实例。
- `generate_feature_attributions`：可选项。模型评估作业是否应生成特征归因。如果未指定，默认为False。

**该流水线大约需要2小时才能完成。**

In [ ]:
job = model.evaluate(
    prediction_type="regression",
    target_field_name="Age",
    gcs_source_uris=[DATA_SOURCE],
    generate_feature_attributions=True,
)

print("Waiting model evaluation is in process")
job.wait()

在上一步的结果中，点击生成的链接，查看在Cloud Console中的运行情况。

在用户界面中，当您点击节点时，许多管道DAG节点会展开或折叠。下面是DAG的部分展开视图（点击图像查看更大的版本）。

获取模型评估结果

评估流程完成后，运行下面的单元格来打印评估指标。

In [ ]:
model_evaluation = job.get_model_evaluation()

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[
            0
        ]  # ['artifacts']
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

### 可视化指标

评估流程完成后，运行下面的单元格来可视化评估指标。

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    if (
        i[0] == "meanAbsolutePercentageError"
    ):  # we are not considering MAPE as it is infinite. MAPE is infinite if groud truth is 0 as in our case Age is 0 for some instances.
        continue
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(10, 5))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

获取特征归因

特征归因显示了模型中每个特征对每个给定实例的预测有多大贡献。

了解有关[特征归因](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview#feature_attributions)的更多信息

运行以下单元格以获取特征归因。

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the feature-attribution task
    if (task.task_name == "feature-attribution") and (
        task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
        or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
    ):
        feat_attrs = task.outputs.get("feature_attributions").artifacts[0]
        feat_attrs_gcs_uri = feat_attrs.uri

print(feat_attrs)
print(feat_attrs_gcs_uri)

从获取的云存储URI中获取特征归因值。

In [ ]:
# Load the results
attributions = !gsutil cat $feat_attrs_gcs_uri

# Print the results obtained
attributions = json.loads(attributions[0])
print(attributions)

### 可视化特征归因

使用条形图可视化每个特征所获得的归因。

In [ ]:
data = attributions["explanation"]["attributions"][0]["featureAttributions"]
features = []
attr_values = []
for key, value in data.items():
    features.append(key)
    attr_values.append(value)

plt.figure(figsize=(5, 3))
plt.bar(x=features, height=attr_values)
plt.title("Feature Attributions")
plt.xticks(rotation=90)
plt.ylabel("Attribution value")
plt.show()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

将`delete_bucket`设置为**True**以创建在此笔记本中创建的Cloud Storage存储桶。

In [ ]:
import os

# Delete model resource
model.delete()

# Delete the dataset resource
dataset.delete()

# Delete the training job
train_job.delete()

# Delete the evaluation pipeline
job.delete()

# Delete Cloud Storage objects
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI